In [1]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 33.5 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595742 sha256=110102353248a96df43bdf44acd36685ccaa07c72f84371c0006ccc9b43d63a0
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 153 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 37.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-qbdg2adx
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-qbdg2adx
     |████████████████████████████████| 132 kB 37.1 MB/s 
     |████████████████████████████████| 4.5 MB 55.5 MB/s 
     |████████████████████████████████| 8.9 MB 1.7 MB/s 
     |████████████████████████████████| 79 kB 10.2 MB/s 
     |████████████████████████████████| 127 kB 69.9 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=279abf4ac65089d7c94c862961e8fd7005a322c1af6954824a6534e68db7d36e
  Stored in directory: /tmp/pip-ephem-wheel-cache-q9gqo3am/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling url

In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-3_5ol051/kobert-tokenizer_0d9141b6b0af4c07aee395adcfbfc27e
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-3_5ol051/kobert-tokenizer_0d9141b6b0af4c07aee395adcfbfc27e
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=5f68656c615b25ff3078e8371309e4ec062b4144b9827041ee318727d97398f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-5sopg93o/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import BertForSequenceClassification, BertConfig

In [7]:
device = torch.device('cuda:0')

In [8]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
  print(torch.cuda.get_device_name(i))

1
Tesla T4


In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
%cd gdrive/MyDrive/

/content/gdrive/MyDrive


In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv('Music.csv')
df

,id,artist_name,song_name,lyrics,label
0,1,CHIMMI (취미),Touch,오렌지 물들은\nMy lips lipstick\n수줍은듯한 코랄 빛\nMy chee...,0
1,2,커피소년,놀이터,나만 설레었던 그 밤에\n또 설레발쳤던 그 밤에\n내 맘을 앗아간 그곳에\n아직도 ...,0
2,3,웨스턴 카잇 (Western Kite),좀비,창백한 얼굴 핏기 없는 입술\n나 긴장했나 봐 니 앞에서\n넌 멀뚱히 나만 쳐다보고...,0
3,4,공기남,파란섬 (feat. 1ho & Chan),날씨가 좋아요 함께 떠날까요\n별을 보며 설레는 대화를 해볼까요\n좀 더 다가와 줘...,0
4,5,스무살,"걷자, 집앞이야 (Feat. 주예인)",hey you 지금 나와\none step two step\nand three st...,0
...,...,...,...,...,...
623,624,H.O.T.,빛 (Hope),늘 함께 있어\n소중한 걸 몰랐던 거죠\n언제나 나와 함께 있어준\n소중한 사람들을...,2
624,625,김보경 (NEON),혼자라고 생각말기 (슈퍼스타 K5 송희진 오디션 참가곡),지치지 않기 포기하지 않기\n어떤 힘든 일에도 늘 이기기\n너무 힘들 땐 너무 지칠...,2
625,626,태연 (TAEYEON),내게 들려주고 싶은 말 (Dear Me),날 놓아줘\n숨 쉴 수 있게\n이 스쳐 가는\n풍경 속에 자유롭게\n노래하듯 나에게...,2
626,627,방탄소년단,EPILOGUE : Young Forever,막이 내리고 나는 숨이 차\n복잡해진 마음 숨을 내쉰다\n오늘 뭐 실수는 없었었나\...,2


In [12]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [13]:
from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
# model = AutoModel.from_pretrained("skt/kobert-base-v1")

In [14]:
df

,id,artist_name,song_name,lyrics,label
0,1,CHIMMI (취미),Touch,오렌지 물들은\nMy lips lipstick\n수줍은듯한 코랄 빛\nMy chee...,0
1,2,커피소년,놀이터,나만 설레었던 그 밤에\n또 설레발쳤던 그 밤에\n내 맘을 앗아간 그곳에\n아직도 ...,0
2,3,웨스턴 카잇 (Western Kite),좀비,창백한 얼굴 핏기 없는 입술\n나 긴장했나 봐 니 앞에서\n넌 멀뚱히 나만 쳐다보고...,0
3,4,공기남,파란섬 (feat. 1ho & Chan),날씨가 좋아요 함께 떠날까요\n별을 보며 설레는 대화를 해볼까요\n좀 더 다가와 줘...,0
4,5,스무살,"걷자, 집앞이야 (Feat. 주예인)",hey you 지금 나와\none step two step\nand three st...,0
...,...,...,...,...,...
623,624,H.O.T.,빛 (Hope),늘 함께 있어\n소중한 걸 몰랐던 거죠\n언제나 나와 함께 있어준\n소중한 사람들을...,2
624,625,김보경 (NEON),혼자라고 생각말기 (슈퍼스타 K5 송희진 오디션 참가곡),지치지 않기 포기하지 않기\n어떤 힘든 일에도 늘 이기기\n너무 힘들 땐 너무 지칠...,2
625,626,태연 (TAEYEON),내게 들려주고 싶은 말 (Dear Me),날 놓아줘\n숨 쉴 수 있게\n이 스쳐 가는\n풍경 속에 자유롭게\n노래하듯 나에게...,2
626,627,방탄소년단,EPILOGUE : Young Forever,막이 내리고 나는 숨이 차\n복잡해진 마음 숨을 내쉰다\n오늘 뭐 실수는 없었었나\...,2


In [15]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [16]:
data_list = []
for lyrics, label in zip(df['lyrics'], df['label']):
  data = []
  data.append(lyrics)
  data.append(str(label))

  data_list.append(data)

In [17]:
data_list

[['오렌지 물들은\nMy lips lipstick\n수줍은듯한 코랄 빛\nMy cheek touch touch\n한껏 들뜬 나의 View\n위에 살짝 걸쳐 Mascara\n적당한 거리에 날 비춰주면\n이내 넌 날 향해 웃어\nTouch you\nDo you love it\n온통 물들어 버리게\nDare you\n해가 지고 있네\nTouch you\n피어올라\n나는 오늘\nYellow butterfly\n네 맘 사이사이를 가로지를래\n뻣뻣한 데님처럼 막\n그렇게 굴지는 마\nI know 내가 퍼져버렸을걸\n한껏 들뜬 나의 View\n위에 살짝 걸쳐 Mascara\n적당한 거리에 날 비춰주면\n이내 넌 날 향해 웃어\nTouch you\nDo you love it\n온통 물들어 버리게\nDare you\n해가 지고 있네\nTouch you\n놀랄 틈조차 주지 않는 걸\n깊숙이 피어난 is you\nis you is you is you\nTouch you\nDo you love it\n온통 물들어 버리게\nDare you\n해가 지고 있네\nTouch you',
  '0'],
 ['나만 설레었던 그 밤에\n또 설레발쳤던 그 밤에\n내 맘을 앗아간 그곳에\n아직도 내 맘 있는 그곳에\n또 널 찾아가 난 그곳에 있네\n내 발걸음이 내 맘 같아서\n내 맘은 너의 향길 맡아서\n여기 있다고 말하면 어떻게 보일까\n아님 아무 말 말고 그냥 기다릴까\n혹시나 해서 마주칠까 하는 생각에\n바보같이 여길 또 서성이네\n여기 있다고 말하면 뭐라 생각할까\n아님 오버인 걸까 내 맘 알아줄까\n널 보고 싶단 말을 못하고\n이리저리 돌려서 말하고\n또 니 곁을 맴도는 나야\n또 널 찾아가 난 그곳에 있네\n내 발걸음이 내 맘 같아서\n내 맘은 너의 향길 맡아서\n여기 있다고 말하면 어떻게 보일까\n아님 아무 말 말고 그냥 기다릴까\n혹시나 해서 마주칠까 하는 생각에\n바보같이 여길 또 서성이네\n여기 있다고 말하면 뭐라 생각할까\n아님 오버인 걸까 내 맘 알아줄까\n널 보고 싶단 말을

In [18]:
data_list[0]

['오렌지 물들은\nMy lips lipstick\n수줍은듯한 코랄 빛\nMy cheek touch touch\n한껏 들뜬 나의 View\n위에 살짝 걸쳐 Mascara\n적당한 거리에 날 비춰주면\n이내 넌 날 향해 웃어\nTouch you\nDo you love it\n온통 물들어 버리게\nDare you\n해가 지고 있네\nTouch you\n피어올라\n나는 오늘\nYellow butterfly\n네 맘 사이사이를 가로지를래\n뻣뻣한 데님처럼 막\n그렇게 굴지는 마\nI know 내가 퍼져버렸을걸\n한껏 들뜬 나의 View\n위에 살짝 걸쳐 Mascara\n적당한 거리에 날 비춰주면\n이내 넌 날 향해 웃어\nTouch you\nDo you love it\n온통 물들어 버리게\nDare you\n해가 지고 있네\nTouch you\n놀랄 틈조차 주지 않는 걸\n깊숙이 피어난 is you\nis you is you is you\nTouch you\nDo you love it\n온통 물들어 버리게\nDare you\n해가 지고 있네\nTouch you',
 '0']

In [26]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [20]:
tok = tokenizer.tokenize
train_data = BERTDataset(data_list, 0, 1, tok, vocab, max_len, True, False)

In [21]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [22]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [23]:
model = BERTClassifier(bertmodel, dr_rate = 0.5).to(device)

In [24]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [27]:
train_history=[]
loss_history=[]
for e in range(num_epochs):
  train_acc = 0.0
  model.train()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
    optimizer.zero_grad()
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
         
    #print(label.shape,out.shape)
    loss = loss_fn(out, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()  # Update learning rate schedule
    train_acc += calc_accuracy(out, label)
    if batch_id % log_interval == 0:
      print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
      train_history.append(train_acc / (batch_id+1))
      loss_history.append(loss.data.cpu().numpy())
  print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.9110676646232605 train acc 0.5
epoch 1 train acc 0.6698317307692307


  0%|          | 0/10 [00:00<?, ?it/s]

...

epoch 48 batch id 1 loss 0.08036182075738907 train acc 1.0
epoch 48 train acc 0.9453125


  0%|          | 0/10 [00:00<?, ?it/s]

epoch 49 batch id 1 loss 0.06394415348768234 train acc 1.0
epoch 49 train acc 0.9484375


  0%|          | 0/10 [00:00<?, ?it/s]

epoch 50 batch id 1 loss 0.0848550945520401 train acc 0.984375
epoch 50 train acc 0.95


In [ ]:
torch.save(model, 'KoBERT_노래가사_2.pt')
torch.save(model.state_dict(), './model_state_dict_2.pt')
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, './all_2.tar')

In [ ]:
PATH = './'
torch.save(model, PATH + 'koBERT_노래가사.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

In [28]:
def new_softmax(a) : 
    c = np.max(a) # 최댓값
    exp_a = np.exp(a-c) # 각각의 원소에 최댓값을 뺀 값에 exp를 취한다. (이를 통해 overflow 방지)
    sum_exp_a = np.sum(exp_a)
    y = (exp_a / sum_exp_a) * 100
    return np.round(y, 3)

In [33]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("사랑이")
            elif np.argmax(logits) == 1:
                test_eval.append("이별이")
            elif np.argmax(logits) == 2:
                test_eval.append("위로를")
          

        print(">> 당신에게 " + test_eval[0])

In [35]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 나 죽고싶어


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 당신에게 위로를


하고싶은 말을 입력해주세요 : 나 울고싶어
>> 당신에게 위로를


하고싶은 말을 입력해주세요 : 나 똥싸고싶어
>> 당신에게 사랑이




KeyboardInterrupt: ignored

---

# gluonnlp

In [ ]:
from gluonnlp.data import SentencepieceTokenizer
from kobert import get_tokenizer
tok_path = get_tokenizer()
sp = SentencepieceTokenizer(tok_path)

/content/gdrive/MyDrive/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
tokenized_data = []

for i in range(0, len(data_list)):
  tokenized_data.append(sp(data_list[i][0]))

tokenized_data